In [ ]:
from selenium import webdriver 
from pymongo import MongoClient
from bs4 import BeautifulSoup
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pymongo
import requests
from time import sleep
import urllib.request
import os

In [ ]:
# Pilote selenium Chrome 
browser = webdriver.Chrome(r'C:\chromedriver.exe')

In [ ]:
# Go to the url
browser.get('https://mbasic.facebook.com/')

In [ ]:
#target username and password
username = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys("53292112") # please insert your user_name here or you can skip this block and insert them manually
password.clear()
password.send_keys("*******") # please insert your password here or you can skip this block and insert them manually

#target the login button and click it
login_btn = browser.find_element("xpath",'//*[@id="login_form"]/ul/li[3]/input')
sleep(3)


login_btn.click()

In [ ]:
#Skip remember popup
login_btn = browser.find_element("xpath",'//*[@id="root"]/table/tbody/tr/td/div/div[3]/a')
login_btn.click()

In [ ]:
#Search
search = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='query']")))

#enter desired Hashtag ! 
#you can change to any hashtag but sometimes you need to change things later some html code are not structured
search.clear()
search.send_keys("#football") 
#search button click
login_btn = browser.find_element("xpath",'//*[@id="header"]/form/table/tbody/tr/td[3]/input')

login_btn.click()

# BeautifulSoup
# cet outil nous permettra de parser les pages html et extraire des données

In [ ]:
list_dir="C:\\"
default_dir = os.path.join(list_dir,"C:/Users/fzaea/Desktop/Scraping/Img") #create a 2 folder on your desktop and change the names also the user name
opener = urllib.request.build_opener()
urllib.request.install_opener(opener)

In [ ]:
posts = browser.find_element("xpath",'//*[@id="root"]/div[1]')
links = posts.find_elements("link text",'Actualité intégrale')
pubs = []
for link in links:
    pub = {}
    page_content = requests.get(link.get_attribute('href')).content
    soup = BeautifulSoup(page_content, 'html.parser')
    #get post text
    pub['text'] = soup.p.text
    #get image url
    pub['image'] = soup.img["src"]
    img_url = soup.img["src"]
    filename = os.path.join(default_dir, img_url.split("/")[-1])
    img_data = opener.open(img_url)
    f = open(filename,"wb")
    f.write(img_data.read())
    f.close()
    pubs.append(pub)

In [ ]:
pubs

# MongoDB
# le text de la publication et l'image vont etre stockés dans une DB Nosql MongoDB

In [ ]:
#MongoDB connection put your connection to verify
client = pymongo.MongoClient("mongodb://localhost:27017")
mydb = client["facebook"]
mycol = mydb["posts"]

#insert the list of dicts
mycol.insert_many(pubs)

#Show a sample 
mycol.find_one()